### Plan of Attack:

1. Get list of new songs after a specific date
2. Get data needed for those songs
3. Create parquet table
4. Save to S3

In [1]:
!pip install spotipy

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import os
import pandas as pd
import numpy as np
%run '../../credentials/settings.py'
from collections import defaultdict

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))

In [3]:
new_releases = sp.new_releases(country='US')

### list items we want from each album, will need to figure out how to get track ids from each album (even if an artist releases a single their is an "album" associated)
- release_date
- total_tracks
- id
- name
- artist(s)
- artist ids

In [4]:
def album_metadata(album_items):
          
    """
    The function extracts data new_releases json response from Spotipy into a pandas df
    
    """
    album_data = {}
    album_data['album_id'] = []
    album_data['release_date'] = []
    album_data['total_tracks']  = []
    album_data['artist_id_1'] = []
    album_data['artist_ids'] = []
    album_data['artist_names'] = []
    
    for album_item in album_items:
    
        # map 1:1 data for each record
        album_data['album_id'].append(album_item['id'])
        album_data['release_date'].append(album_item['release_date'])
        album_data['total_tracks'].append(album_item['total_tracks'])

        # get artist(s) data
        artist_lst = album_item['artists']
        artist_ids = [x['id'] for x in artist_lst]
        album_data['artist_id_1'].append(artist_ids[0])
        album_data['artist_ids'].append(str(artist_ids))
        album_data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(album_data,orient='columns')

In [5]:
#test it
album_df = album_metadata(new_releases['albums']['items'])
album_df

,album_id,release_date,total_tracks,artist_id_1,artist_ids,artist_names
0,4hY1TaqQxOL2aBAWvAI111,2021-08-20,1,1zNqQNIdeOUZHb8zbZRFMX,"['1zNqQNIdeOUZHb8zbZRFMX', '5ZS223C6JyBfXasXxr...","['Swae Lee', 'Jhené Aiko']"
1,7jMo8dhMIorHoLlzTisjJy,2021-08-13,1,56oDRnqbIiwx4mymNEv7dS,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']"
2,2TvfE8CY37OQIPVGcWYpEA,2021-08-12,1,2o5jDhtHVPhrJdv3cEQ99Z,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']"
3,5D8Rdb09BkmHscEGSWAlA6,2021-08-13,1,3PhoLpVuITZKcymswpck5b,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']"
4,00fcDjvEq4elj756TESO0c,2021-08-13,1,78rUTD7y6Cy67W1RVzYs7t,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress']
5,1qMFjBarjO2xD15BwXZguD,2021-08-13,11,0C0XlULifJtAgn6ZNCW2eu,['0C0XlULifJtAgn6ZNCW2eu'],['The Killers']
6,6vqNFOfIgqe0l2Ui47YSyO,2021-08-13,6,2cFrymmkijnjDg9SS92EPM,['2cFrymmkijnjDg9SS92EPM'],['blackbear']
7,4ih8LPpqdYPezAMaPAHJYT,2021-08-07,1,46SHBwWsqBkxI7EeeBEQG7,"['46SHBwWsqBkxI7EeeBEQG7', '45TgXXqMDdF8BkjA83...","['Kodak Black', 'Rod Wave']"
8,3k4lUXBZxjUGFFQvy8PsHn,2021-08-12,1,6TIYQ3jFPwQSRmorSezPxX,['6TIYQ3jFPwQSRmorSezPxX'],['Machine Gun Kelly']
9,4pWo1MFBvVAfhjL4aPZFaD,2021-08-13,1,1vyhD5VmyZ7KMfW5gqLgo5,['1vyhD5VmyZ7KMfW5gqLgo5'],['J Balvin']


### figure out how to get track IDs from each of album avoe of the above

In [6]:
def tracks_from_album(df):
      
    """
    The function uses Spotipy to fetch track data for all albums released in album_df.
    
    """
    data = {}
    data['album_id'] = []
    data['track_id'] = []
    data['track_name'] = []
    data['artist_id_1'] = []
    data['artist_id_2'] = []
    data['artist_ids'] = []
    data['artist_names'] = []
    data['explicit'] = []
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    for i,x in df.iterrows():
        album_id = x['album_id']
        track_data = sp.album_tracks(album_id)
        items = track_data['items']

        for item in items:

            # map 1:1 data for each record
            data['album_id'].append(album_id)
            data['track_id'].append(item['id'])
            data['track_name'].append(item['name'])
            data['explicit'].append(item['explicit'])

            # get artist(s) data
            artist_lst = item['artists']
            artist_ids = [x['id'] for x in artist_lst]
            data['artist_id_1'].append(artist_ids[0])
            if len(artist_ids) > 1:
                data['artist_id_2'].append(artist_ids[1])
            else:
                data['artist_id_2'].append('')
            data['artist_ids'].append(str(artist_ids))
            data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(data,orient='columns')

In [7]:
%%time
tracks_df = tracks_from_album(album_df)
tracks_df.head()

CPU times: user 67.1 ms, sys: 13.1 ms, total: 80.2 ms
Wall time: 1.49 s


,album_id,track_id,track_name,artist_id_1,artist_id_2,artist_ids,artist_names,explicit
0,4hY1TaqQxOL2aBAWvAI111,4ZB2yAZyPHI6SHdFPPT9sa,In The Dark (with Jhené Aiko),1zNqQNIdeOUZHb8zbZRFMX,5ZS223C6JyBfXasXxrRqOk,"['1zNqQNIdeOUZHb8zbZRFMX', '5ZS223C6JyBfXasXxr...","['Swae Lee', 'Jhené Aiko']",False
1,7jMo8dhMIorHoLlzTisjJy,6KgtcmCF9Ky68XC7ezxl3s,Rumors (feat. Cardi B),56oDRnqbIiwx4mymNEv7dS,4kYSro6naA4h99UJvo89HB,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']",True
2,2TvfE8CY37OQIPVGcWYpEA,0bI7K9Becu2dtXK1Q3cZNB,Don't Be Shy,2o5jDhtHVPhrJdv3cEQ99Z,790FomKkXshlbRYZFtlgla,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']",False
3,5D8Rdb09BkmHscEGSWAlA6,6zSpb8dQRaw0M1dK8PBwQz,Cold Heart - PNAU Remix,3PhoLpVuITZKcymswpck5b,6M2wZ9GZgrQXHCFfjv46we,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']",False
4,00fcDjvEq4elj756TESO0c,6OTKVgVpVaVjhRLYizPJKA,Just for me,78rUTD7y6Cy67W1RVzYs7t,,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress'],False


### merge to get release date of album

In [8]:
df = tracks_df.merge(album_df[['album_id', 'release_date']],how='left',on='album_id')
df.head()

,album_id,track_id,track_name,artist_id_1,artist_id_2,artist_ids,artist_names,explicit,release_date
0,4hY1TaqQxOL2aBAWvAI111,4ZB2yAZyPHI6SHdFPPT9sa,In The Dark (with Jhené Aiko),1zNqQNIdeOUZHb8zbZRFMX,5ZS223C6JyBfXasXxrRqOk,"['1zNqQNIdeOUZHb8zbZRFMX', '5ZS223C6JyBfXasXxr...","['Swae Lee', 'Jhené Aiko']",False,2021-08-20
1,7jMo8dhMIorHoLlzTisjJy,6KgtcmCF9Ky68XC7ezxl3s,Rumors (feat. Cardi B),56oDRnqbIiwx4mymNEv7dS,4kYSro6naA4h99UJvo89HB,"['56oDRnqbIiwx4mymNEv7dS', '4kYSro6naA4h99UJvo...","['Lizzo', 'Cardi B']",True,2021-08-13
2,2TvfE8CY37OQIPVGcWYpEA,0bI7K9Becu2dtXK1Q3cZNB,Don't Be Shy,2o5jDhtHVPhrJdv3cEQ99Z,790FomKkXshlbRYZFtlgla,"['2o5jDhtHVPhrJdv3cEQ99Z', '790FomKkXshlbRYZFt...","['Tiësto', 'KAROL G']",False,2021-08-12
3,5D8Rdb09BkmHscEGSWAlA6,6zSpb8dQRaw0M1dK8PBwQz,Cold Heart - PNAU Remix,3PhoLpVuITZKcymswpck5b,6M2wZ9GZgrQXHCFfjv46we,"['3PhoLpVuITZKcymswpck5b', '6M2wZ9GZgrQXHCFfjv...","['Elton John', 'Dua Lipa', 'PNAU']",False,2021-08-13
4,00fcDjvEq4elj756TESO0c,6OTKVgVpVaVjhRLYizPJKA,Just for me,78rUTD7y6Cy67W1RVzYs7t,,['78rUTD7y6Cy67W1RVzYs7t'],['PinkPantheress'],False,2021-08-13


### get audio features for every track

In [9]:
def audio_features_from_tracks(track_ids):
  
    """
    The function uses Spotipy to fetch audio features and metadata for the specified track_ids.
    
    """
    
    song_data = defaultdict()
    audio_features = sp.audio_features(track_ids)
    
    for i,song in enumerate(audio_features):
        if i == 0:
            for key, value in song.items():
                song_data[key] = [value]
        else:
            for key, value in song.items():
                song_data[key].append(value)
    
    df = pd.DataFrame(song_data)
    df = df.rename(columns={'id':'track_id'})
    df = df.drop(columns=['track_href','analysis_url','time_signature','uri','type'])
                
    return df
    

In [10]:
%%time
track_ids = df.track_id.tolist()
audio_df = audio_features_from_tracks(track_ids)
audio_df.head()

CPU times: user 16.6 ms, sys: 66 µs, total: 16.7 ms
Wall time: 184 ms


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id,duration_ms
0,0.693,0.646,6,-6.583,1,0.0883,0.364,0.000000,0.1080,0.640,98.029,4ZB2yAZyPHI6SHdFPPT9sa,161164
1,0.827,0.731,4,-5.524,0,0.0880,0.130,0.000000,0.4180,0.607,118.992,6KgtcmCF9Ky68XC7ezxl3s,172833
2,0.770,0.787,11,-3.748,0,0.0583,0.198,0.000000,0.2620,0.513,119.986,0bI7K9Becu2dtXK1Q3cZNB,140500
3,0.796,0.798,1,-6.312,1,0.0317,0.034,0.000042,0.0952,0.942,116.032,6zSpb8dQRaw0M1dK8PBwQz,202735
4,0.778,0.628,0,-7.844,1,0.0618,0.705,0.001930,0.2990,0.698,133.048,6OTKVgVpVaVjhRLYizPJKA,115801


### get genres for every artist

In [18]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                       client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))

artists = sp.artists(artist_ids)

In [19]:
len(artists)

1

In [20]:
artist_ids

['1zNqQNIdeOUZHb8zbZRFMX',
 '56oDRnqbIiwx4mymNEv7dS',
 '2o5jDhtHVPhrJdv3cEQ99Z',
 '3PhoLpVuITZKcymswpck5b',
 '78rUTD7y6Cy67W1RVzYs7t',
 '0C0XlULifJtAgn6ZNCW2eu',
 '2cFrymmkijnjDg9SS92EPM',
 '46SHBwWsqBkxI7EeeBEQG7',
 '6TIYQ3jFPwQSRmorSezPxX',
 '1vyhD5VmyZ7KMfW5gqLgo5',
 '7z5WFjZAIYejWy0NI5lv4T',
 '4iMO20EPodreIaEl8qW66y',
 '7gXb99Sf9nNmpNYeAgIQFG',
 '0ErzCpIMyLcjPiwT4elrtZ',
 '5JMLG56F1X5mFmWNmS0iAp',
 '6DgP9otnZw5z6daOntINxp',
 '0WK3H9OErSn5zKOkOV5egm',
 '59oA5WbbQvomJz2BuRG071',
 '2VDsDEKCdNm7c7Ve43Z9gw',
 '2WqEbbet6L2ndAbvhRVb2S']

In [24]:
len(artists['artists'])

20

In [25]:
artists['artists'][0].keys()

dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])

In [26]:
def artist_metadata(artist_ids):
      
    """
    The function uses Spotipy to fetch artist data and returns as df given list of artist ids.
    
    """
    data = {}
    data['artist_id'] = []
    data['genres'] = []
    data['popularity'] = []
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    
    artists = sp.artists(artist_ids)

    for artist in artists['artists']:

        # map 1:1 data for each record
        data['artist_id'].append(artist['id'])
        data['genres'].append(artist['genres'])
        data['popularity'].append(artist['popularity'])
    
    return pd.DataFrame.from_dict(data,orient='columns')

In [27]:
artist_ids = df.artist_id_1.unique().tolist()
artist_df = artist_metadata(artist_ids)
artist_df.head()

,artist_id,genres,popularity
0,1zNqQNIdeOUZHb8zbZRFMX,"[melodic rap, trap]",84
1,56oDRnqbIiwx4mymNEv7dS,"[dance pop, escape room, minnesota hip hop, po...",80
2,2o5jDhtHVPhrJdv3cEQ99Z,"[big room, brostep, dance pop, dutch edm, edm,...",86
3,3PhoLpVuITZKcymswpck5b,"[glam rock, mellow gold, piano rock]",85
4,78rUTD7y6Cy67W1RVzYs7t,[],80


### main function for loop
# re copying all functions below

In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import os
import pandas as pd
import numpy as np
%run '../../credentials/settings.py'
from collections import defaultdict

In [2]:
def album_metadata(album_items):  
    """
    The function extracts data new_releases json response from Spotipy into a pandas df
    
    """
    album_data = {}
    album_data['album_id'] = []
    album_data['release_date'] = []
    album_data['total_tracks']  = []
    album_data['artist_id_1'] = []
    album_data['artist_ids'] = []
    album_data['artist_names'] = []
    
    for album_item in album_items:
    
        # map 1:1 data for each record
        album_data['album_id'].append(album_item['id'])
        album_data['release_date'].append(album_item['release_date'])
        album_data['total_tracks'].append(album_item['total_tracks'])

        # get artist(s) data
        artist_lst = album_item['artists']
        artist_ids = [x['id'] for x in artist_lst]
        album_data['artist_id_1'].append(artist_ids[0])
        album_data['artist_ids'].append(str(artist_ids))
        album_data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(album_data,orient='columns')

In [30]:
def tracks_from_album(df,sp):
      
    """
    The function uses Spotipy to fetch track data for all albums released in album_df.
    
    """
    data = {}
    data['album_id'] = []
    data['track_id'] = []
    data['track_name'] = []
    data['artist_id_1'] = []
    data['artist_id_2'] = []
    data['artist_ids'] = []
    data['artist_names'] = []
    data['explicit'] = []
    
#     sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
#                                                            client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    for i,x in df.iterrows():
        album_id = x['album_id']
        track_data = sp.album_tracks(album_id)
        items = track_data['items']

        for item in items:

            # map 1:1 data for each record
            data['album_id'].append(album_id)
            data['track_id'].append(item['id'])
            data['track_name'].append(item['name'])
            data['explicit'].append(item['explicit'])

            # get artist(s) data
            artist_lst = item['artists']
            artist_ids = [x['id'] for x in artist_lst]
            data['artist_id_1'].append(artist_ids[0])
            if len(artist_ids) > 1:
                data['artist_id_2'].append(artist_ids[1])
            else:
                data['artist_id_2'].append('')
            data['artist_ids'].append(str(artist_ids))
            data['artist_names'].append(str([x['name'] for x in artist_lst]))
    
    return pd.DataFrame.from_dict(data,orient='columns')

In [31]:
def audio_features_from_tracks(track_ids,sp):
  
    """
    The function uses Spotipy to fetch audio features and metadata for the specified track_ids.
    
    """
#     sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
#                                                        client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    
    
    df = pd.DataFrame() # holder of all results
    
    # add limit as you can only pass in 100 audio features at a time
    n = len(track_ids)
    x = n // 100
    limit_lst = [(x+1)*100 for x in range(x)]
    limit_lst.append(n)
    start = 0
    for limit in limit_lst:
        song_data = defaultdict()
        audio_features = sp.audio_features(track_ids[start:limit])
        start+=(limit-start)

        for i,song in enumerate(audio_features):
            if i == 0:
                for key, value in song.items():
                    song_data[key] = [value]
            else:
                for key, value in song.items():
                    song_data[key].append(value)

        df_temp = pd.DataFrame(song_data)
        df_temp = df_temp.rename(columns={'id':'track_id'})
        df_temp = df_temp.drop(columns=['track_href','analysis_url','time_signature','uri','type'])
        df = df.append(df_temp)
                
    return df

In [33]:
def artist_metadata(artist_ids,sp):
      
    """
    The function uses Spotipy to fetch artist data and returns as df given list of artist ids.
    
    """
    data = {}
    data['artist_id'] = []
    data['genres'] = []
    data['popularity'] = []
    
#     sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
#                                                            client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    
    artists = sp.artists(artist_ids)

    for artist in artists['artists']:

        # map 1:1 data for each record
        data['artist_id'].append(artist['id'])
        data['genres'].append(artist['genres'])
        data['popularity'].append(artist['popularity'])
    
    return pd.DataFrame.from_dict(data,orient='columns')

In [53]:
def loop_new_releases(limit,loops):    
    """
    The function extracts the new releases in n loops with limit number of new releases at a time
    
    """
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))
    album_df = pd.DataFrame()
    tracks_df = pd.DataFrame()
    audio_df = pd.DataFrame()
    artist_df = pd.DataFrame()
    
    for i in range(loops):
        # run above functions for each loop
        new_releases = sp.new_releases(country='US',limit=limit,offset=limit*i)
        temp_album_df = album_metadata(new_releases['albums']['items'])
        temp_tracks_df = tracks_from_album(temp_album_df,sp)
        temp_audio_df = audio_features_from_tracks(temp_tracks_df.track_id.tolist(),sp)
        temp_artist_df = artist_metadata(temp_album_df.artist_id_1.unique().tolist(),sp)
        
        # append temp dfs
        album_df = album_df.append(temp_album_df)
        tracks_df = tracks_df.append(temp_tracks_df)
        audio_df = audio_df.append(temp_audio_df)
        artist_df = artist_df.append(temp_artist_df)
        
        
    return album_df, tracks_df, audio_df, artist_df

In [54]:
%%time
album_df, tracks_df, audio_df, artist_df = loop_new_releases(50,2)
for i in [album_df, tracks_df, audio_df, artist_df]:
    print(i.shape)

(100, 6)
(504, 8)
(504, 13)
(100, 3)
CPU times: user 446 ms, sys: 14.2 ms, total: 460 ms
Wall time: 8.65 s


In [47]:
from datetime import datetime
dtobj = datetime.now()
dt_str = dtobj.strftime("%Y%m%d")
print(dt_str)

20210820


In [52]:
## write to csv
names = ['album_df','tracks_df','audio_df','artist_df']
for ind,i in enumerate([album_df, tracks_df, audio_df, artist_df]):
    i.to_csv('../../temp_data/{}.csv'.format(names[ind]+'_'+dt_str))